In [1]:
import numpy as np
import pandas as pd
import os
import radvel
from radvel.plot import orbit_plots, mcmc_plots
from scipy import optimize
import ktwo19
import ktwo19.io
%pylab osx

/Users/petigura/anaconda2/lib/python2.7/site-packages/scipy/stats/_continuous_distns.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _stats


ImportError: cannot import name select_autoescape

In [2]:
df = pd.read_csv('data/photometry/ktwo-everest.tsv',sep='\t')
def masktransits(df):
    tc1 = 813.3837 + 2456000 - 2454833
    per1 = 7.9194
    tphase = np.mod(df.t - tc1 + 0.5 * per1,per1) - 0.5*per1
    df['mask'] = df['mask'] | np.array((np.abs(tphase) < 0.15))
    tc2 = 817.2755 + 2456000 - 2454833 
    per2 = 11.90715
    tphase = np.mod(df.t - tc2 + 0.5 * per2,per2) - 0.5*per2
    df['mask'] = df['mask'] | np.array((np.abs(tphase) < 0.15))
    df = df.query('mask==0')
    return df 
    
plot(df.t,df.f,'.')

In [3]:
# Compute scatter in photmetry
sigma_diff = 1.48 * df.f.diff().abs().median() 
sigma_phot = sigma_diff / np.sqrt(2)
sigma_phot

9.2999753544474508e-05

In [4]:
hnames = [
    'gp_amp', # eta_1; GP variability amplitude
    'gp_explength', # eta_2; GP non-periodic characteristic length
    'gp_per', # eta_3; GP variability period
    'gp_perlength', # eta_4; GP periodic characteristic length
]

nplanets=1
params = radvel.Parameters(nplanets,basis='per tc secosw sesinw k')
params['per1'] = radvel.Parameter(value=7.9207019469,vary=False)
params['tc1'] = radvel.Parameter(value=1980.3770766406,vary=False)
params['sesinw1'] = radvel.Parameter(value=0.,vary=False) # fix eccentricity = 0
params['secosw1'] = radvel.Parameter(value=0.,vary=False)
params['k1'] = radvel.Parameter(value=0,vary=False)
params['dvdt'] = radvel.Parameter(value=0,vary=False)
params['curv'] = radvel.Parameter(value=0,vary=False)


params['gp_amp'] = radvel.Parameter(value=0.001)
params['gp_explength'] = radvel.Parameter(value=10)
params['gp_per'] = radvel.Parameter(value=20.4,vary=True)
params['gp_perlength'] = radvel.Parameter(value=0.4)

photmodel = radvel.model.RVModel(params)

dftrain = df.iloc[::10]
like = radvel.likelihood.GPLikelihood(
    photmodel, dftrain.t, dftrain.f, sigma_phot, hnames, suffix='_phot',kernel_name="QuasiPer"
)

# Add in instrument parameters
like.params['gamma_phot'] = radvel.Parameter(value=np.mean(dftrain.f),vary=True)
like.params['jit_phot'] = radvel.Parameter(value=0.001,vary=True)

gppost = radvel.posterior.Posterior(like)
gppost.priors += [radvel.prior.Jeffreys('gp_amp', 250e-6, 0.01)]
gppost.priors += [radvel.prior.Jeffreys('jit_phot', 1e-6, 1e-3)]
#gppost.priors += [radvel.prior.Gaussian('gp_explength', gp_explength_mean, gp_explength_unc)]
#gppost.priors += [radvel.prior.Gaussian('gp_per', gp_per_mean, gp_per_unc)]
#gppost.priors += [radvel.prior.Gaussian('gp_perlength', gp_perlength_mean, gp_perlength_unc)]

res = optimize.minimize(
    gppost.neglogprob_array, gppost.get_vary_params(), method='Powell',
    options=dict(maxiter=200, maxfev=100000, xatol=1e-8)
)

print(gppost)


/Users/petigura/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:44: OptimizeWarning: Unknown solver options: xatol
/Users/petigura/anaconda2/lib/python2.7/site-packages/scipy/optimize/optimize.py:1927: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/Users/petigura/anaconda2/lib/python2.7/site-packages/scipy/optimize/optimize.py:1932: RuntimeWarning: invalid value encountered in absolute
  tmp2 = numpy.abs(tmp2)


parameter                     value      vary
per1                         7.9207      False
tc1                         1980.38      False
secosw1                           0      False
sesinw1                           0      False
k1                                0      False
dvdt                              0      False
curv                              0      False
gp_amp                         0.01       True
gp_explength               0.254081       True
gp_per                  1.41906e+08       True
gp_perlength                10.7517       True
gamma_phot                 0.984895       True
jit_phot                      0.001       True

Priors
------
Jeffrey's prior on gp_amp, min=0.00025, max=0.01
Jeffrey's prior on jit_phot, min=1e-06, max=0.001



In [5]:
clf()

like = radvel.likelihood.GPLikelihood(
    photmodel, df.t, df.f, sigma_phot, hnames, suffix='_phot',kernel_name="QuasiPer"
)

like.params['gamma_phot'] = radvel.Parameter(value=np.mean(dftrain.f),vary=True)
like.params['jit_phot'] = radvel.Parameter(value=0.001,vary=True)

gppost2 = radvel.posterior.Posterior(like)
import copy
gppost2.params = copy.deepcopy(gppost.params)




In [21]:
hnames = [
    'gp_amp', # eta_1; GP variability amplitude
    'gp_explength', # eta_2; GP non-periodic characteristic length
    'gp_per', # eta_3; GP variability period
    'gp_perlength', # eta_4; GP periodic characteristic length
]

nplanets=1
params = radvel.Parameters(nplanets,basis='per tc secosw sesinw k')
params['per1'] = radvel.Parameter(value=7.9207019469,vary=False)
params['tc1'] = radvel.Parameter(value=1980.3770766406,vary=False)
params['sesinw1'] = radvel.Parameter(value=0.,vary=False) # fix eccentricity = 0
params['secosw1'] = radvel.Parameter(value=0.,vary=False)
params['k1'] = radvel.Parameter(value=0,vary=False)
params['dvdt'] = radvel.Parameter(value=0,vary=False)
params['curv'] = radvel.Parameter(value=0,vary=False)


params['gp_amp'] = radvel.Parameter(value=0.004243)
params['gp_explength'] = radvel.Parameter(value=33.984398)
params['gp_per'] = radvel.Parameter(value=20.387807,vary=True)
params['gp_perlength'] = radvel.Parameter(value=0.485900)



photmodel = radvel.model.RVModel(params)


,close all
fig = figure(figsize=(8,3))

df = pd.read_csv('data/photometry/ktwo-everest.tsv',sep='\t')
df = df.query('mask==0 and not (2015 < t < 2017)')
dfm = masktransits(df)
like = radvel.likelihood.GPLikelihood(
    photmodel, dfm.t, dfm.f, sigma_phot, hnames, suffix='_phot',kernel_name="QuasiPer"
)

# Add in instrument parameters
like.params['gamma_phot'] = radvel.Parameter(value=1.001307,vary=True)
like.params['jit_phot'] = radvel.Parameter(value=0.000679,vary=True)

gppost2 = radvel.posterior.Posterior(like)

plot(df.t,df.f,'.',ms=2,color='k',label='photometry')

t = np.linspace(df.t.iloc[0],df.t.iloc[-1],1000)
mu_pred, std_pred  = gppost2.likelihood.predict(t)
plot(t,mu_pred + gppost2.params['gamma_phot'].value,'r',label='GP')
fig.set_tight_layout(True)
xlabel('BJD - 2454833')
ylabel('Normalized Flux')
fill_between(t,mu_pred + gppost2.params['gamma_phot'].value + std_pred ,mu_pred + gppost2.params['gamma_phot'].value - std_pred,color='r',alpha=0.3)
grid()
legend()
fig.savefig('fig_photometry-gp.pdf')



In [ ]:
#chains = radvel.mcmc(gppost,nrun=1000,ensembles=3)

/Users/petigura/anaconda2/lib/python2.7/site-packages/emcee/ensemble.py:335: RuntimeWarning: invalid value encountered in subtract
  lnpdiff = (self.dim - 1.) * np.log(zz) + newlnprob - lnprob0
/Users/petigura/anaconda2/lib/python2.7/site-packages/emcee/ensemble.py:336: RuntimeWarning: invalid value encountered in greater
  accept = (lnpdiff > np.log(self._random.rand(len(lnpdiff))))
/Users/petigura/anaconda2/lib/python2.7/site-packages/emcee/ensemble.py:335: RuntimeWarning: invalid value encountered in subtract
  lnpdiff = (self.dim - 1.) * np.log(zz) + newlnprob - lnprob0
/Users/petigura/anaconda2/lib/python2.7/site-packages/emcee/ensemble.py:336: RuntimeWarning: invalid value encountered in greater
  accept = (lnpdiff > np.log(self._random.rand(len(lnpdiff))))
/Users/petigura/anaconda2/lib/python2.7/site-packages/emcee/ensemble.py:335: RuntimeWarning: invalid value encountered in subtract
  lnpdiff = (self.dim - 1.) * np.log(zz) + newlnprob - lnprob0
/Users/petigura/anaconda2/lib/py

7500/150000 (5.0%) steps complete; Running 250.49 steps/s; Mean acceptance rate = 1.1%; Min Tz = 128.9; Max G-R = 1.024      
Discarding burn-in now that the chains are marginally well-mixed



/Users/petigura/anaconda2/lib/python2.7/site-packages/emcee/ensemble.py:335: RuntimeWarning: invalid value encountered in subtract
  lnpdiff = (self.dim - 1.) * np.log(zz) + newlnprob - lnprob0
/Users/petigura/anaconda2/lib/python2.7/site-packages/emcee/ensemble.py:336: RuntimeWarning: invalid value encountered in greater
  accept = (lnpdiff > np.log(self._random.rand(len(lnpdiff))))
/Users/petigura/anaconda2/lib/python2.7/site-packages/emcee/ensemble.py:335: RuntimeWarning: invalid value encountered in subtract
  lnpdiff = (self.dim - 1.) * np.log(zz) + newlnprob - lnprob0
/Users/petigura/anaconda2/lib/python2.7/site-packages/emcee/ensemble.py:336: RuntimeWarning: invalid value encountered in greater
  accept = (lnpdiff > np.log(self._random.rand(len(lnpdiff))))
/Users/petigura/anaconda2/lib/python2.7/site-packages/emcee/ensemble.py:335: RuntimeWarning: invalid value encountered in subtract
  lnpdiff = (self.dim - 1.) * np.log(zz) + newlnprob - lnprob0
/Users/petigura/anaconda2/lib/py

In [ ]:
print  chains.describe().ix[['mean','std']].T.to_string(float_format='%.6f')

In [ ]:
Corner = mcmc_plots.CornerPlot(gppost, chains) # posterior distributions
Corner.plot()